In [28]:
#Code generated by Anna Cassidy (with some help from ChatGPT) 06/10/2025
#Last edited by AC on 01/28/2026
#Full directions can be found in the ReadMe document
#Track duration processing and filtering.


import os
import re
import glob
import pandas as pd
from typing import Optional

# ------------------------------------------------------------------
# 1)  Where things live
# ------------------------------------------------------------------
base_dirs = [
    r"C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1",
    r"C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250919-PtK1 EB1-GFP cells H2B-mCherry Replicate 2",
    r"C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250926-PtK1 EB1-GFP cells H2B-mCherry Replicate 3"
]

phase_folders = [
    "Interphase",
    "Prophase",
    "Prometaphase",
    "Metaphase",
    "Anaphase",
    "2-Daughter-Cells"
]

spec_file = (
    r"C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\PtK1-Pooled_Imaging_Specs_Summary.xlsx"
)

# ------------------------------------------------------------------
# 2)  Build lookup: image-ID (last 3 digits) → seconds per frame
# ------------------------------------------------------------------
spec_df = pd.read_excel(spec_file)

def last3digits(s: str) -> Optional[str]:
    """Return the final 3-digit ID in the string (e.g. '070')."""
    m = re.search(r"(\d{3})(?!.*\d)", s)
    return m.group(1) if m else None

id_to_spf: dict[str, float] = {}
for _, row in spec_df.iterrows():
    img_id = last3digits(str(row["Filename"]))
    spf = row.get("Frame Rate (s/frame)")
    if img_id and pd.notna(spf):
        id_to_spf[img_id] = float(spf)

print(f"✅ Loaded {len(id_to_spf)} image → frame-rate mappings\n")

# ------------------------------------------------------------------
# 3)  Process every Points-file-*.xlsx
# ------------------------------------------------------------------
summary_rows = []  # Collect per-file statistics

for replicate_dir in base_dirs:
    if not os.path.isdir(replicate_dir):
        print(f"[SKIP] Replicate folder missing: {replicate_dir}")
        continue

    replicate_name = os.path.basename(replicate_dir)
    print(f"\n📂 Processing replicate: {replicate_name}")

    for phase in phase_folders:
        phase_dir = os.path.join(replicate_dir, phase)
        if not os.path.isdir(phase_dir):
            print(f"   [SKIP] Phase folder missing: {phase}")
            continue

        # find every “…\sample\RESULTS\Points-file-*.xlsx”
        pattern = os.path.join(phase_dir, "*", "RESULTS", "Points-file-*.xlsx")
        matches = glob.glob(pattern)
        if not matches:
            print(f"   [INFO] No Points files found in {phase}")
            continue

        for points_path in matches:
            points_name = os.path.basename(points_path)
            sample_id = points_name.removeprefix("Points-file-").removesuffix(".xlsx")
            img_id = last3digits(sample_id.lower())

            # 3a.  look up frame rate
            spf = id_to_spf.get(img_id)
            if spf is None:
                print(f"   [WARN] No frame-rate found for {points_name} (ID {img_id})")
                continue

            print(f"   → {phase}/{points_name}   (spf = {spf:.4f} s)")

            # 3b.  Load file
            try:
                df = pd.read_excel(points_path)
            except Exception as e:
                print(f"      [ERROR] Could not read file: {e}")
                continue

            if df.empty or "Y" not in df.columns:
                print(f"      [SKIP] File empty or missing 'Y' column.")
                continue

            # -----------------------------------------------------------
            # 3c.  Compute track duration
            # -----------------------------------------------------------
            df["Track Duration"] = ""
            sheet2_rows, sheet3_rows = [], []
            count_ge4, count_lt4 = 0, 0

            for i in range(0, len(df) - 1, 2):
                try:
                    track_duration = (df.loc[i + 1, "Y"] - df.loc[i, "Y"]) * spf
                except (KeyError, ValueError, TypeError):
                    continue

                df.loc[i, "Track Duration"] = track_duration

                if track_duration >= 4:
                    target = sheet2_rows
                    count_ge4 += 1
                else:
                    target = sheet3_rows
                    count_lt4 += 1

                target.append(df.iloc[i].tolist())
                target.append(df.iloc[i + 1].tolist())

            sheet2_df = pd.DataFrame(sheet2_rows, columns=df.columns)
            sheet3_df = pd.DataFrame(sheet3_rows, columns=df.columns)

            # -----------------------------------------------------------
            # 3d.  Write outputs
            # -----------------------------------------------------------
            out_sorted = points_path.replace(".xlsx", "_sorted.xlsx")
            try:
                with pd.ExcelWriter(out_sorted) as xl:
                    df.to_excel(xl, index=False, sheet_name="Original Data")
                    sheet2_df.to_excel(xl, index=False, sheet_name="Tracks ≥ 4")
                    sheet3_df.to_excel(xl, index=False, sheet_name="Tracks < 4")
            except Exception as e:
                print(f"      [ERROR] Could not write {out_sorted}: {e}")
                continue

            out_filtered = os.path.join(
                os.path.dirname(points_path), "Points_file_4sORmore.xlsx"
            )

            try:
                sheet2_df.drop(columns=["Track Duration"]).to_excel(
                    out_filtered, index=False, sheet_name="Tracks ≥ 4"
                )
            except Exception as e:
                print(f"      [ERROR] Could not write {out_filtered}: {e}")
                continue

            print(f"      ✔  Saved:\n         {out_sorted}\n         {out_filtered}")

            # Add to summary
            total = count_ge4 + count_lt4
            pct_ge4 = (count_ge4 / total * 100) if total > 0 else 0
            summary_rows.append({
                "Replicate": replicate_name,
                "Phase": phase,
                "Sample": sample_id,
                "Tracks ≥4s": count_ge4,
                "Tracks <4s": count_lt4,
                "Total Tracks": total,
                "% ≥4s": round(pct_ge4, 1)
            })

# ------------------------------------------------------------------
# 4)  Summarize results
# ------------------------------------------------------------------
if not summary_rows:
    print("\n⚠️  No track data processed — check your input folders.")
else:
    summary_df = pd.DataFrame(summary_rows)

    # Phase + replicate summary
    phase_summary = (
        summary_df.groupby(["Replicate", "Phase"], as_index=False)
        [["Tracks ≥4s", "Tracks <4s", "Total Tracks"]]
        .sum()
    )
    phase_summary["% ≥4s"] = (
        phase_summary["Tracks ≥4s"] / phase_summary["Total Tracks"] * 100
    ).round(1)

    # ------------------------------------------------------------------
    # Save both sheets into one Excel summary
    # ------------------------------------------------------------------
    summary_out = os.path.join(os.path.dirname(spec_file), "Track_Summary_All.xlsx")
    with pd.ExcelWriter(summary_out) as xl:
        summary_df.to_excel(xl, index=False, sheet_name="Per Sample")
        phase_summary.to_excel(xl, index=False, sheet_name="Per Phase Summary")

    print("\n📊 SUMMARY (per phase):")
    print(phase_summary.to_string(index=False))
    print(f"\n✅ Full summary saved to:\n   {summary_out}")


✅ Loaded 43 image → frame-rate mappings


📂 Processing replicate: 20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1
   → Interphase/Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1030.xlsx   (spf = 0.1471 s)
      ✔  Saved:
         C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Interphase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1030\RESULTS\Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1030_sorted.xlsx
         C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Interphase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1030\RESULTS\Points_file_4sORmore.xlsx
   → Interphase/Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1030_sorted.xlsx   (spf = 0.1471 s)
      ✔  Saved:
         C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Interphase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_5

      ✔  Saved:
         C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Prophase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046\RESULTS\Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046_sorted_sorted.xlsx
         C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Prophase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046\RESULTS\Points_file_4sORmore.xlsx
   → Prophase/Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046_sorted_sorted.xlsx   (spf = 0.1526 s)
      [ERROR] Could not write C:\Users\annao\OneDrive - Vanderbilt\Desktop\To_be_analyzed\20250918-PtK1 EB1-GFP cells H2B-mCherry Replicate 1\Prophase\100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046\RESULTS\Points-file-100x_PtK1-EB1-GFP-4ugtryp60p_488EB1_561H2B_1046_sorted_sorted_sorted.xlsx: [Errno 2] No such file or directory: 'C:\\Users\\annao\\OneDrive - Vanderbilt\\Desktop\\To_be_analyze

KeyboardInterrupt: 